# Vectorize Images with Siamese Network

In [1]:
import os
import glob

import numpy as np
from PIL import Image
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torchvision
from torch.nn import functional as nn_F
from torchvision.transforms import functional as tran_F

from logos_recognition.utils import save_df

#### Parameters

In [2]:
logos_path = os.path.join(os.environ['DATASETS'], 'logos', 'logos', 'all_crops')
exemplars_path = os.path.join(os.environ['DATASETS'], 'logos', 'exemplars')
model_filename = os.path.join(os.environ['HKT'], 'logodetect',
                              'models', 'siamese_embedding.pth')
img_size = 100
model_out = 345
device = 'cuda:2'
net_type = '_siamese'

#### Load model

In [3]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, model_out)
checkpoint = torch.load(model_filename)
model.load_state_dict(checkpoint['state_dict'])
model.eval().to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
# Define the transform:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((img_size, img_size)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465),
                                     (0.2023, 0.1994, 0.2010)),
])

In [5]:
def calc_nn_feat(image, transform, model):
    image = transform(image).unsqueeze(0).to(device, dtype=torch.float)
    features = model(image)
    features = nn_F.normalize(features, p=2, dim=1)
    return features.squeeze().detach().cpu().numpy()

In [6]:
def vectorize_images(path, model_out, transform, model):
    # Extract vectors:
    filenames = glob.glob(path + '/*.jpg')
    vectors = np.zeros((len(filenames), model_out))
    for idx, img_path in enumerate(tqdm(filenames)):
        image = Image.open(img_path).convert('RGB')
        vectors[idx, :] = calc_nn_feat(image, transform, model)
    return vectors, filenames

#### Process logos

In [7]:
logos_vectors, logos_filenames = vectorize_images(
    logos_path, model_out, transform, model)
save_df(logos_vectors, logos_filenames, logos_path, net_type)

100%|██████████| 76017/76017 [06:16<00:00, 201.64it/s]


#### Process exemplars

In [8]:
exemplars_vectors, exemplars_filenames = vectorize_images(
    exemplars_path, model_out, transform, model)
save_df(exemplars_vectors, exemplars_filenames, exemplars_path, net_type)

100%|██████████| 352/352 [00:02<00:00, 131.50it/s]
